## CH01.03. **LangSmith**

#### (1) **LangSmith** 추적 설정

- **LLM 애플리케이션 개발, 모니터링 및 테스트를 위한 플랫폼**
- 추적 기능 = LLM 동작을 이해하기 위한 강력한 도구
- 문제점 추적에 도움이 됨
    - 예상치 못한 최종 결과
    - 에이전트가 루핑되는 이유
    - 체인이 예상보다 느린 이유
    - 에이전트가 각 단계에서 사용한 토큰 수
- 추적 단위 : 프로젝트, 1개의 실행데 대한 세부 단계별 추적 가능

- **LangSmith API Key 발급**
  
- 환경변수(`.env`)에 LangSmith 키 설정
    - `LANGCHAIN_TRACING_V2` = `ture`로 설정 시 추적 시작
    - `LANGCHAIN_ENDPOINT` = `https://api.smith.langchain.com` (변경 X)
    - `LANGCHAIN_API_KEY` = 위에서 발급받은 API Key
    - `LANGCHAIN_PROJECT` = 프로젝트명 기입 -> 해당 프로젝트 그룹으로 모든 실행(Run)이 추적됨

#### (2) **Jupyter Notebook (혹은 코드)에서 추적 활성화**

- 환경 변수 설정

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()               # true

True

#### (3) langchain-teddynote

- `langchain-teddynote` package 사용하기

- LangSmith 추적 설정 [https://smith.langchain.com]
- .env 파일에 LANGCHAIN_API_KEY를 입력하기

- 설치 (둘 중 하나)
    - 셀 : `!pip install -qU langchain-teddynote`
    - 터미널 : `pip install langchain-teddynote`   ✓

In [ ]:
# 터미널에서 langchain-teddynote 패키지 설치
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

<small>

* 셀 출력 
    * LangSmith 추적을 시작합니다.
    * [프로젝트명]
    * CH01-Basic
</small>

        - 추적을 원하지 않을 경우 아래와 같이 추적 끄기

In [ ]:
from langchain_teddynote import logging

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith("랭체인 튜토리얼 프로젝트", set_enable=False)         # LangSmith 추적을 하지 않습니다.

--------

## CH01.04. **GEMINI API 사용**하기

#### (1) GEMINI

- Google의 Large Language Model(LLM)

- 객체를 생성할 때 지정할 수 있는 옵셥

    - `temperature`
        - 모델이 텍스트를 생성할 때 얼마나 "창의적"이거나 "무작위적"으로 응답할지를 제어
        - 0.0과 2.0 사이의 값으로 설정
        - 높은 값은 더 예측 불가능하고 다양한 출력을, 낮은 값은 더 집중되고 결정론적인 출력을 생성
        - 예시: 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듦

    - `max_tokens` (혹은 `maxOutTokens`)
        - 모델이 생성할 수 있는 응답 텍스트의 최대 길이를 토큰 단위로 제한
        - 정수 값으로 설정 / 모델별로 최대 허용 토큰 수 다름
        - 예시: Gemini 2.5 Pro/Flash는 최대 65,536 출력 토큰
  
    - 참고 문서
        - Google Cloud Generative AI Docs - Text generation parameters [https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/content-generation-parameters?hl=ko]
        - Gemini API Reference - GenerationConfig [https://cloud.google.com/vertex-ai/generative-ai/docs/reference/rest/v1/GenerationConfig]



- `model_name`: 적용 가능한 모델 리스트
  - `gemini-2.5-pro`:(입력)오디오, 이미지, 동영상, 텍스트, PDF / (출력) 텍스트
  - `gemini-2.5-flash`: (입력)오디오, 이미지, 동영상, 텍스트 / (출력) 텍스트
  - `gemini-2.5-flash-lite`: (입력) 텍스트, 이미지, 동영상, 오디오 / (출력) 텍스트
  
    ![Gemini 모델 개요](../01_basic/images/gemini_models.png)

  - `gemini-2.5-flash-preview-native-audio-dialog&gemini-2.5-flash-exp-native-audio-thinking-dialog`: (입력) 오디오, 동영상, 텍스트 / (출력) 텍스트 + 오디오
  - `gemini-2.5-flash-preview-tts`: (입력) 텍스트 / (출력) 오디오
  - `gemini-2.5-pro-preview-tts`: (입력) 텍스트 / (출력) 오디오
  - `gemini-2.0-flash`: (입력) 오디오, 이미지, 동영상, 텍스트 / (출력) 텍스트
  - `gemini-2.0-flash-preview-image-generation`: (입력) 오디오, 이미지, 동영상, 텍스트 / (출력) 텍스트, 이미지
  - `gemini-2.0-flash-lite`: (입력) 오디오, 이미지, 동영상, 텍스트 / (출력) 텍스트

- 링크: https://ai.google.dev/gemini-api/docs?hl=ko

In [ ]:
# Gemini version

import os                                           # 환경 변수 접근을 위해 추가
from dotenv import load_dotenv                      # .env 파일 로드를 위해 추가

#from google import genai 
#from google.genai import types
from datetime import datetime
from langchain_google_genai import ChatGoogleGenerativeAI


# 환경 변수 로드
load_dotenv()

# 객체 생성
# model_name 대신 model 파라미터를 사용하고, Gemini 모델명을 지정합
# temperature는 동일하게 사용
''''
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents=["Explain how AI works"],
    config=types.GenerateContentConfig(
        temperature=0.1
    )
)
'''

# Create LLM class
llm = ChatGoogleGenerativeAI(
    model= "gemini-2.5-flash-lite",  # 모델 이름을 지정합니다.
    temperature=1.0,
    max_retries=2,
    google_api_key=os.environ.get("GOOGLE_API_KEY"),
)


# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

# 선택 사항: API 키가 제대로 로드되었는지 확인 (보안을 위해 일부 가림)
# print(f"GOOGLE_API_KEY: {os.environ.get('GOOGLE_API_KEY')[:-15]}" + "*" * 15 if os.environ.get('GOOGLE_API_KEY') else "API 키가 로드되지 않았습니다.")
#  ㄴ-> 정상 작동 확인 

<small>

* 셀 출력
    * [답변]: content='대한민국의 수도는 **서울**입니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []} id='run--87475bfb-837d-4181-b493-789ad6507bb0-0' usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}}

#### (2) 답변의 형식(AI Message)

In [8]:
# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
response = llm.invoke(question)

In [ ]:
response

<small>

* 셀 출력 
    * AIMessage(content='대한민국의 수도는 **서울**입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--98cb0fcb-dc23-4201-a0d2-31a7cd5e344c-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}})

In [ ]:
response.content

<small>

* 셀 출력
    * '대한민국의 수도는 **서울**입니다.'

In [ ]:
response.response_metadata

<small>

* 셀 출력
    * {'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
    * 'finish_reason': 'STOP',
    * 'model_name': 'gemini-2.5-flash-lite',
    * 'safety_ratings': []}

#### (3) LogProb 활성화

- 주어진 텍스트에 대한 모델의 **토큰 확률의 로그 값**
- 토큰 = 문장을 구성하는 개별 단어나 문자 등의 요소
- 확률 = **모델이 그 토큰을 예측할 확률**

In [ ]:
# 객체 생성
llm_with_logprob = ChatGoogleGenerativeAI(
    temperature=0.1,                        # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,                        # 최대 토큰수
    model="gemini-2.5-flash-lite",          # 모델명
    
    # generation_config를 사용하여 로그 확률을 활성화
    # response_logprobs=True: 선택된 토큰의 로그 확률 반환
    # logprobs=5: 상위 5개 대체 토큰의 로그 확률도 함께 반환
    generation_config={
        "response_logprobs": True,
        "logprobs": 5 # 1부터 20까지의 정수 (원하는 대체 토큰 수)
    }
)

In [19]:
# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
response = llm_with_logprob.invoke(question)

In [20]:
# 응답 출력 예시 (모델 응답은 response.content에 있음)
print(f"[답변]: {response.content}")

[답변]: 대한민국의 수도는 **서울**입니다.


In [21]:
# 결과 출력
print("\n--- 추가 응답 메타데이터 (로그 확률 포함 가능성) ---")
print(response.response_metadata)


--- 추가 응답 메타데이터 (로그 확률 포함 가능성) ---
{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}


In [ ]:
import os
from dotenv import load_dotenv

from datetime import datetime
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

# Google API 키 설정
api_key = os.getenv("GOOGLE_API_KEY")


# 모델 초기화 및 generation_config 설정
llm = ChatGoogleGenerativeAI(
        model ="gemini-2.5-pro",        # 모델명은 필요에 따라 변경 가능
        generation_config={
            "temperature": 0.1,
            "max_output_tokens": 2048,
            "response_logprobs": True,      # 선택된 토큰의 로그 확률 반환 활성화
            "logprobs": 5,                  # 상위 5개 대체 토큰의 로그 확률도 함께 반환 (1-20 사이)
            "google_api_key" : api_key,           # API 키 설정
        }
)

# 질의 내용
question = "대한민국의 수도는 어디인가요?"

# 질의 및 응답 받기
response = llm.invoke(question)

# 응답 텍스트 출력
print(f"[답변]: {response.text}")



<small>

* 셀 출력
    * [답변]: <bound method BaseMessage.text of AIMessage(content='대한민국의 수도는 **서울**입니다.
    * 정식 명칭은 **서울특별시**이며, 대한민국의 정치, 경제, 사회, 문화의 중심지입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': []}, id='run--1ca2b14e-ab85-417d-85fb-577e7777ed01-0', usage_metadata={'input_tokens': 10, 'output_tokens': 445, 'total_tokens': 455, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 409}})>

#### (4) 스트리밍 출력

- 스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용

In [30]:
# 스트림 방식으로 질의
# answer 에 스트리밍 답변의 결과를 받습니다.
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")

In [28]:
# 스트리밍 방식으로 각 토큰을 출력합니다. (실시간 출력)
for token in answer:
    print(token.content, end="", flush=True)

<small>

* 셀 출력
    * 대한민국에는 정말 아름다운 관광지가 많지만, 그중에서도 많은 사람들에게 사랑받는 대표적인 10곳과 주소를 알려드릴게요!

        **1. 제주도 (Jeju Island)**

        *   **설명:** 한국의 하와이라고 불릴 만큼 아름다운 자연경관을 자랑하는 섬입니다. 화산 활동으로 만들어진 독특한 지형과 푸른 바다, 다양한 해양 액티비티를 즐길 수 있습니다.
        *   **주소:** 제주특별자치도 제주시

        **2. 경주 (Gyeongju)**

        *   **설명:** 신라 시대의 천년 고도로, 수많은 유적과 문화재가 살아 숨 쉬는 곳입니다. 불국사, 석굴암, 첨성대 등 역사적인 명소들을 둘러보며 과거로 시간 여행을 떠날 수 있습니다.
        *   **주소:** 경상북도 경주시

        **3. 서울 (Seoul)**

        *   **설명:** 대한민국의 수도이자 역사와 현대가 공존하는 도시입니다. 경복궁, 창덕궁 등 고궁과 현대적인 빌딩, 활기찬 쇼핑 거리, 맛집들이 어우러져 다채로운 매력을 선사합니다.
        *   **주소:** 서울특별시

        **4. 부산 (Busan)**

        *   **설명:** 한국 제2의 도시이자 아름다운 해변 도시입니다. 해운대, 광안리 해수욕장 등 멋진 해변과 함께 감천문화마을, 자갈치시장 등 독특한 볼거리와 먹거리가 풍부합니다.
        *   **주소:** 부산광역시

        **5. 전주 한옥마을 (Jeonju Hanok Village)**

        *   **설명:** 한국 전통 가옥인 한옥이 잘 보존된 곳으로, 고즈넉한 분위기 속에서 한국의 전통 문화를 체험할 수 있습니다. 맛있는 전주 비빔밥과 길거리 음식을 맛보는 재미도 쏠쏠합니다.
        *   **주소:** 전라북도 전주시 완산구 기린대로 99

        **6. 강릉 (Gangneung)**

        *   **설명:** 동해안의 아름다운 해변과 함께 오죽헌, 선교장 등 역사적인 명소, 그리고 맛있는 커피로 유명한 도시입니다. 특히 여름철에는 시원한 바다를 즐기기 위해 많은 사람들이 찾습니다.
        *   **주소:** 강원특별자치도 강릉시

        **7. 속초 (Sokcho)**

        *   **설명:** 설악산 국립공원과 동해 바다를 동시에 즐길 수 있는 곳입니다. 설악산의 웅장한 자연과 속초항의 싱싱한 해산물을 맛볼 수 있습니다.
        *   **주소:** 강원특별자치도 속초시

        **8. 안동 하회마을 (Andong Hahoe Village)**

        *   **설명:** 조선 시대 양반 마을의 모습을 그대로 간직하고 있는 곳입니다. 낙동강이 마을을 휘감아 도는 지형과 전통 가옥들이 어우러져 아름다운 풍경을 자아냅니다.
        *   **주소:** 경상북도 안동시 풍천면 하회종가길 40

        **9. 담양 죽녹원 (Damyang Juknokwon)**

        *   **설명:** 울창한 대나무 숲길을 걸으며 심신을 치유할 수 있는 곳입니다. 시원한 바람과 함께 대나무가 만들어내는 푸른 터널은 잊지 못할 경험을 선사합니다.
        *   **주소:** 전라남도 담양군 담양읍 죽녹원로 119

        **10. 남이섬 (Nami Island)**

        *   **설명:** 드라마 촬영지로 유명해진 아름다운 섬입니다. 메타세쿼이아길, 은행나무길 등 계절마다 다른 매력을 뽐내는 산책로와 다양한 예술 작품들을 감상할 수 있습니다.
        *   **주소:** 강원특별자치도 춘천시 남산면 남이섬길 1

        * 이 외에도 대한민국에는 숨겨진 보석 같은 아름다운 관광지가 정말 많습니다. 여행 계획 세우실 때 참고하시면 좋을 것 같습니다! 즐거운 여행 되세요!

In [ ]:
from langchain_teddynote.messages import stream_response

# 스트림 방식으로 질의
# answer 에 스트리밍 답변의 결과를 받습니다.
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")
stream_response(answer)

<small>

* 셀 출력
    
    * 네, 대한민국에서 빼어난 아름다움을 자랑하는 관광지 10곳을 주소와 함께 소개해 드리겠습니다. 자연경관, 역사 유적, 문화 명소 등을 골고루 포함했습니다.

---

  * 대한민국 아름다운 관광지 10선

  * **1. 서울 경복궁 (Gyeongbokgung Palace)**
  조선 왕조 제1의 법궁으로, 웅장하고 아름다운 한국의 궁궐 건축미를 느낄 수 있는 곳입니다. 사계절 모두 다른 매력을 뽐내며, 특히 한복을 입고 방문하면 더욱 특별한 경험을 할 수 있습니다.
    *   **주소:** 서울특별시 종로구 사직로 161

  * **2. 제주 성산일출봉 (Seongsan Ilchulbong Peak, Jeju)**
  유네스코 세계자연유산으로 등재된 곳으로, 바다 위로 솟아오른 거대한 화산체입니다. 정상에서 바라보는 일출과 주변 바다 풍경이 장관을 이룹니다.
    *   **주소:** 제주특별자치도 서귀포시 성산읍 성산리 1

  * **3. 경주 불국사 (Bulguksa Temple, Gyeongju)**
  신라 시대 불교 예술의 정수를 보여주는 사찰로, 유네스코 세계문화유산입니다. 다보탑과 석가탑의 정교한 아름다움과 조화로운 가람 배치가 인상적입니다.
    *   **주소:** 경상북도 경주시 불국로 385

  * **4. 전주 한옥마을 (Jeonju Hanok Village)**
  700여 채의 전통 한옥이 밀집된 곳으로, 고즈넉한 골목길을 산책하며 한국의 전통문화를 체험할 수 있습니다. 맛있는 길거리 음식도 이곳의 큰 매력입니다.
    *   **주소:** 전라북도 전주시 완산구 기린대로 99 (한옥마을 관광안내소)

  * **5. 부산 감천문화마을 (Gamcheon Culture Village, Busan)**
  산비탈을 따라 계단식으로 들어선 파스텔톤의 집들이 만들어내는 풍경이 아름다워 '한국의 산토리니'라고 불립니다. 골목마다 숨겨진 예술 작품을 찾는 재미가 쏠쏠합니다.
    *   **주소:** 부산광역시 사하구 감내2로 203 (감천문화마을 안내센터)

  * **6. 강원 설악산 국립공원 (Seoraksan National Park, Gangwon)**
  대한민국을 대표하는 명산으로, 기암괴석과 맑은 계곡, 울창한 숲이 어우러져 사계절 내내 절경을 이룹니다. 특히 가을 단풍이 아름답기로 유명합니다.
    *   **주소:** 강원특별자치도 속초시 설악산로 833 (설악산 국립공원사무소)

  * **7. 보성 녹차밭 (Boseong Green Tea Fields)**
  끝없이 펼쳐진 초록빛 녹차밭이 언덕을 따라 물결치는 모습은 한 폭의 그림 같습니다. 상쾌한 녹차 향을 맡으며 산책로를 걷는 것만으로도 힐링이 됩니다.
    *   **주소:** 전라남도 보성군 보성읍 녹차로 763-67 (대한다원)

  * **8. 순천만습지 (Suncheon Bay Wetland Reserve)**
  광활한 갯벌과 끝없이 펼쳐진 갈대밭이 장관을 이루는 대한민국 최대의 연안습지입니다. 특히 해 질 녘 S자 물길을 따라 붉게 물드는 노을은 잊지 못할 감동을 선사합니다.
    *   **주소:** 전라남도 순천시 순천만길 513-25

  * **9. 수원 화성 (Suwon Hwaseong Fortress)**
  조선 후기 건축 기술의 백미로 꼽히는 성곽으로, 유네스코 세계문화유산입니다. 성곽길을 따라 걸으며 아름다운 성문과 주변 경관을 감상할 수 있습니다.
    *   **주소:** 경기도 수원시 팔달구 정조로 825 (화성행궁)

  * **10. 서울 N서울타워 (N Seoul Tower)**
  서울의 중심인 남산 정상에 위치한 랜드마크로, 타워 전망대에서 서울의 화려한 전경을 360도로 조망할 수 있습니다. 낮의 풍경도 멋지지만, 특히 야경이 아름답습니다.
    *   **주소:** 서울특별시 용산구 남산공원길 105

  ---

  * 이 외에도 대한민국에는 숨겨진 아름다운 명소들이 많으니, 즐거운 여행 계획을 세우시는 데 도움이 되기를 바랍니다

#### (5) 프롬프트 캐싱

- 참고 링크: https://ai.google.dev/gemini-api/docs/caching?hl=ko&_gl=1*11rtxhv*_up*MQ..*_ga*ODU5OTIzMjk2LjE3NTM1OTYyOTA.*_ga_P1DBVKWT6V*czE3NTM1OTYyOTAkbzEkZzAkdDE3NTM1OTYyOTAkajYwJGwwJGgxNDc3MDI2Mg..&lang=python 
  

- 프롬프트 캐싱 기능을 활용하면 반복하여 동일하게 입력으로 들어가는 토큰에 대한 비용을 아낄 수 있음
- 캐싱에 활용할 토큰은 **고정된 PREFIX**를 주는 것 권장
  
- 아래의 예시에서는 `<PROMPT_CACHING>` 부분에 고정된 토큰을 주어 캐싱을 활용하는 방법을 설명

In [29]:
from langchain_teddynote.messages import stream_response

very_long_prompt = """
당신은 매우 친절한 AI 어시스턴트 입니다. 
당신의 임무는 주어진 질문에 대해 친절하게 답변하는 것입니다.
아래는 사용자의 질문에 답변할 때 참고할 수 있는 정보입니다.
주어진 정보를 참고하여 답변해 주세요.

<WANT_TO_CACHE_HERE>
#참고:
**Prompt Caching**
모델 프롬프트에는 시스템 프롬프트 및 일반적인 지침과 같이 반복되는 콘텐츠가 포함되는 경우가 많습니다.
Gemini API는 이러한 반복되는 콘텐츠에 대한 **컨텍스트 캐싱(Context Caching)** 메커니즘을 제공하여, 초기부터 프롬프트를 처리하는 것보다 더 효율적이고 빠르게 만들고 비용을 절감할 수 있습니다.

**Gemini API는 다음 두 가지 캐싱 메커니즘을 제공하며, 각 방식별 지원 모델이 다릅니다.**

1.  **암시적 캐싱 (Implicit Caching)**
    * **자동 활성화**: 개발자가 별도 설정 없이 자동으로 캐싱 혜택을 받습니다.
    * **지원 모델**:
        * `gemini-2.5-pro`
        * `gemini-2.5-flash`
    * **비용 절감**: 캐시된 토큰은 할인된 요율(일반 입력 토큰 비용의 25%)로 청구되며, 캐시 쓰기나 저장 비용은 발생하지 않습니다.
    * **최소 토큰 수**: 캐싱이 적용되려면 `gemini-2.5-flash`의 경우 최소 1,024개 입력 토큰, `gemini-2.5-pro`의 경우 최소 2,048개 입력 토큰이 필요합니다.
    * **캐시 히트율 높이기**: 프롬프트 시작 부분에 크고 일반적인 콘텐츠를 배치하고, 짧은 시간 내에 유사한 접두사로 요청을 보내는 것이 좋습니다.

2.  **명시적 캐싱 (Explicit Caching)**
    * **수동 설정**: 개발자가 직접 캐시를 생성하고 관리하며, 비용 절감을 보장하고자 할 때 유용합니다.
    * **지원 모델**:
        * `gemini-2.5-pro`
        * `gemini-2.5-flash`
        * `gemini-2.0-pro`
        * `gemini-2.0-flash`
        * `gemini-2.0-flash-lite`
        * (대부분의 Gemini 2.0 및 2.5 모델)
    * **작동 방식**: 일부 콘텐츠를 모델에 한 번 전달하고 입력 토큰을 캐시한 다음, 후속 요청에서 이 캐시된 토큰을 참조하여 사용합니다.
    * **비용**: 캐시 토큰 사용 시 할인된 요금이 적용되지만, **캐시 저장 기간에 따라 비용이 발생**합니다.
    * **최소 캐시 크기**: `gemini-2.5-flash`, `gemini-2.0-flash`, `gemini-2.0-flash-lite`는 1,024 토큰, `gemini-2.5-pro` 및 `gemini-2.0-pro`는 2,048 토큰이 최소입니다.
    * **캐시 수명 (TTL)**: 토큰이 자동으로 삭제되기 전에 캐시가 존재할 시간을 선택할 수 있습니다. 설정하지 않으면 기본 TTL은 1시간입니다.

**컨텍스트 캐싱을 사용해야 하는 경우:**
* 광범위한 시스템 안내를 제공하는 챗봇
* 긴 동영상 파일의 반복 분석
* 대규모 문서 세트에 대한 반복 쿼리
* 빈번한 코드 저장소 분석 또는 버그 수정

**주요 고려사항:**
* **모델 인식**: 모델은 캐시된 토큰과 일반 입력 토큰을 구분하지 않습니다. 캐시된 콘텐츠는 프롬프트의 접두사 역할을 합니다.
* **할당량 및 요금**: 컨텍스트 캐싱에 특별한 비율 또는 사용량 제한은 없습니다. `GenerateContent`의 표준 비율 제한이 적용되며, 캐시된 토큰도 토큰 한도에 포함됩니다. 캐시된 토큰 수는 캐시 서비스의 작업 및 `GenerateContent` 호출 시 `usage_metadata`에 반환됩니다.
* **데이터 프라이버시**: 프롬프트 캐시는 조직 간에 공유되지 않습니다. 동일한 조직의 구성원만 동일한 프롬프트의 캐시에 접근할 수 있습니다.
* **출력 영향 없음**: 프롬프트 캐싱은 출력 토큰 생성이나 API의 최종 응답에 영향을 미치지 않습니다. 캐싱 여부와 관계없이 생성되는 출력은 동일합니다.
* **수동 캐시 삭제**: 현재 수동 캐시 삭제 기능은 제공되지 않습니다. 최근에 사용되지 않은 프롬프트는 캐시에서 자동으로 제거됩니다. (일반적으로 비활성 상태 5~10분 후, 비수기에는 최대 1시간까지 지속될 수 있음)
* **추가 비용 없음**: 캐싱은 자동으로 이루어지며, 캐싱 기능을 사용하기 위한 명시적인 조치나 추가 비용은 없습니다. (명시적 캐싱의 경우 저장 기간에 따른 비용 발생 가능)

</WANT_TO_CACHE_HERE>

#Question:
{}

"""

In [ ]:
import os
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage # 필요할 경우

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")


llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",  # 모델명은 필요에 따라 변경 가능
    google_api_key=api_key,
    generation_config={
        "temperature": 0.1,
        "max_output_tokens": 2048,
    }
)

# 답변 요청
answer = llm.invoke(
    very_long_prompt.format("프롬프트 캐싱 기능에 대해 2문장으로 설명하세요"),
    # 캐시를 사용하려면 여기에 config 파라미터가 추가되어야 함
    # 예시: config={"cached_content": cache.name}
    # 이 부분은 이전에 캐시를 생성하는 코드와 연결되어야 함
)

print(f"\n[답변]: {answer.content}")

# 디버깅: answer.response_metadata가 어떤 형태인지 직접 확인하기
print("\n--- 응답 메타데이터 전체 구조 확인 ---")
print(answer.response_metadata)

# 만약 token_usage 정보가 있다면 접근 시도 (구조가 다를 수 있음을 명심!)
if "token_usage" in answer.response_metadata:
    token_usage = answer.response_metadata["token_usage"]
    print(f"\n--- 토큰 사용량 ---")
    print(f"  프롬프트 토큰: {token_usage.get('prompt_tokens', '정보 없음')}")
    print(f"  응답 토큰: {token_usage.get('completion_tokens', '정보 없음')}")
    print(f"  총 토큰: {token_usage.get('total_tokens', '정보 없음')}")
    
    # 캐싱된 토큰은 OpenAI에 특화된 정보이므로, Gemini 응답에서는 없을 가능성이 높습니다.
    # 해당 키가 없을 경우 오류가 발생할 수 있으므로, .get() 메소드를 사용하는 것이 안전합니다.
    # cached_tokens = answer.response_metadata["token_usage"]["prompt_tokens_details"]["cached_tokens"]
    # print(f"캐싱된 토큰: {cached_tokens}")
else:
    print("토큰 사용량 정보가 response_metadata에 포함되어 있지 않습니다.")


<small>

* 셀 출력
    * [답변]: 안녕하세요! Gemini API의 프롬프트 캐싱 기능에 대해 궁금하시군요. 😊
    * 프롬프트 캐싱은 반복되는 프롬프트 내용을 효율적으로 처리하여 응답 속도를 높이고 비용을 절감하는 데 도움을 주는 기능입니다. Gemini API는 개발자가 별도의 설정 없이 자동으로 혜택을 받는 **암시적 캐싱**과 직접 캐시를 관리하여 비용 절감을 보장하는 **명시적 캐싱** 두 가지 방식을 제공합니다.

    * --- 응답 메타데이터 전체 구조 확인 ---
    * {'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}
  
    * 토큰 사용량 정보가 response_metadata에 포함되어 있지 않습니다.

-----

#### (4) 멀티모달 모델(이미지 인식)

- 멀티모달은 여러 가지 형태의 정보(모달)를 통합하여 처리하는 기술이나 접근 방식을 의미합니다. 이는 
- 다음과 같은 다양한 데이터 유형을 포함

    - 텍스트: 문서, 책, 웹 페이지 등의 글자로 된 정보
    - 이미지: 사진, 그래픽, 그림 등 시각적 정보
    - 오디오: 음성, 음악, 소리 효과 등의 청각적 정보
    - 비디오: 동영상 클립, 실시간 스트리밍 등 시각적 및 청각적 정보의 결합

- `Gemini` 모델 = 처음부터 **멀티모달로 설계**
    - 전문 ML모델 학습 없이도 이미지 캡셔닝, 분류, 시각적 질의 응답 등 다양한 이미지 처리 및 컴퓨터 비전 작업 수행 가능
    - `Gemini` 모델 (2.0이상) = **객체 감지** 및 **세그멘테이션** 등 특정 사용 사례에 대한 정확도가 향상

In [ ]:
import os
import requests # 온라인 이미지 다운로드를 위해 requests 라이브러리 추가
from dotenv import load_dotenv
from google import genai
from PIL import Image # Image.open()을 위해 PIL (Pillow) 라이브러리 필요

# .env 파일 로드 (GEMINI_API_KEY 환경 변수 포함)
load_dotenv()

# 환경 변수에서 GEMINI_API_KEY 가져오기
# genai.Client()는 기본적으로 이 환경 변수를 찾습니다.
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY 환경 변수가 설정되지 않았습니다.")

# GenAI 클라이언트 객체 생성
client = genai.Client(api_key=api_key)

# 사용할 Gemini 모델 지정 (Jay님의 요청에 따라 'gemini-2.5-flash-lite' 사용)
# 만약 'gemini-2.5-flash-lite' 모델이 더 이상 제공되지 않거나 문제가 발생할 경우,
# 'gemini-1.5-flash' 또는 'gemini-1.5-pro'와 같은 현재 사용 가능한 최신 모델을 고려해야 할 수 있습니다.
# 현재 시점(2025년 7월)에서 'gemini-2.5-flash-lite'가 유효하다는 전제하에 진행합니다.
MODEL_NAME = 'gemini-2.5-flash'

# --- 이미지 파일 경로 설정 ---
# 이미지 파일은 모두 교재와 똑같이 설정 

# 1. 로컬 이미지 파일 경로
LOCAL_IMAGE_PATH = "./images/sample-image.png"

# 2. 온라인 이미지 URL 
ONLINE_IMAGE_URL = "https://t3.ftcdn.net/jpg/03/77/33/96/360_F_377339633_Rtv9I77sSmSNcev8bEcnVxTHrXB4nRJ5.jpg"

# --- 이미지 로드 함수 ---
def load_image_from_path(image_path):
    """로컬 파일 경로에서 이미지를 로드합니다."""
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"오류: 로컬 이미지 파일 '{image_path}'을(를) 찾을 수 없습니다. 경로를 확인해주세요.")
    return Image.open(image_path)

def load_image_from_url(image_url):
    """온라인 URL에서 이미지를 다운로드하여 로드합니다."""
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status() # HTTP 오류가 발생하면 예외 발생
        return Image.open(response.raw)
    except Exception as e:
        raise ConnectionError(f"오류: 온라인 이미지 '{image_url}'을(를) 로드하는 데 실패했습니다: {e}")

try:
    # --- 로컬 이미지로 멀티모달 스트리밍 ---
    print("\n--- 로컬 이미지로 멀티모달 스트리밍 시작 ---")
    local_image = load_image_from_path(LOCAL_IMAGE_PATH)

    # 모델에 로컬 이미지와 텍스트 프롬프트 전달 (스트리밍)
    local_image_response_chunks = client.models.generate_content_stream(
        model=MODEL_NAME,
        contents=[
            local_image,
            '이 로컬 이미지에 대해 자세히 설명하고, 스트리밍 형식으로 답변해 주세요.'
        ]
    )

    print("[로컬 이미지 답변 (스트리밍)]:")
    for chunk in local_image_response_chunks:
        print(chunk.text, end='') # 스트리밍이므로 한 조각씩 출력
    print("\n--- 로컬 이미지 스트리밍 종료 ---\n")

    # --- 온라인 이미지로 멀티모달 스트리밍 ---
    print("\n--- 온라인 이미지로 멀티모달 스트리밍 시작 ---")
    online_image = load_image_from_url(ONLINE_IMAGE_URL)

    # 모델에 온라인 이미지와 텍스트 프롬프트 전달 (스트리밍)
    online_image_response_chunks = client.models.generate_content_stream(
        model=MODEL_NAME,
        contents=[
            online_image,
            '이 온라인 이미지에 대해 자세히 설명하고, 스트리밍 형식으로 답변해 주세요.'
        ]
    )

    print("[온라인 이미지 답변 (스트리밍)]:")
    for chunk in online_image_response_chunks:
        print(chunk.text, end='') # 스트리밍이므로 한 조각씩 출력
    print("\n--- 온라인 이미지 스트리밍 종료 ---\n")

except FileNotFoundError as e:
    print(f"파일 오류: {e}")
except ConnectionError as e:
    print(f"네트워크 오류: {e}")
except Exception as e:
    print(f"예상치 못한 오류 발생: {e}")

print("모델 사용 완료")


<small>

* 셀 출력
  
![로컬 이미지](images/sample-image.png) 
= 교재와 동일 

    * --- 로컬 이미지로 멀티모달 스트리밍 시작 ---
    * [로컬 이미지 답변 (스트리밍)]:
    * 네, 이 로컬 이미지에 대해 자세히 설명해 드리겠습니다. 마치 실시간으로 스트리밍하듯이요.

---

    * [00:00] 자, 지금 보시는 이미지는 'FIRST OPENAI DEVDAY EVENT'라는 제목을 가진 정보성 그래픽입니다.
    * [00:05] 이벤트 날짜는 '2023년 11월 6일'로 명확하게 표시되어 있네요.
    * [00:10] 이미지 왼쪽 상단에는 주황색 'A' 모양의 로고와 함께 'ASTRA TECHZ'라는 회사 이름, 그리고 'Simplifying Technology'라는 태그라인이 보입니다. 아마 이 정보를 정리한 주체인 것 같습니다.

    * [00:20] 상단 중앙 부분에는 이번 데브데이에서 발표된 다섯 가지 주요 업데이트가 요약되어 있습니다.
    * [00:25] 'GPT 4 Turbo', '128k Tokens' 지원, 'Custom GPTs' 도입, 'Assistant API' 공개, 그리고 'Price Reduction' 즉 가격 인하가 그것들입니다.

    * [00:35] 아래쪽에는 이 모든 정보의 출처가 명시되어 있네요. 'Source OpenAI DevDay Event'라고 되어 있고, YouTube 링크까지 친절하게 달려 있습니다. 'https://www.youtube.com/watch?v=U9mjUlJkhUzk' 이 주소입니다.

    * [00:45] 이제 이미지의 하단으로 내려가 보면, 'MAIN UPDATES SUMMARISED'라는 제목 아래에 매우 상세한 표가 펼쳐져 있습니다.
    * [00:50] 이 표는 왼쪽 열에 업데이트된 기능의 이름이 있고, 가운데 열에는 체크 표시가 되어 있으며, 오른쪽 열에는 해당 기능에 대한 구체적인 설명이나 특징이 적혀 있습니다.

    * [01:00] 하나씩 살펴보겠습니다.
    * [01:02] 첫 번째, 'Token Length'는 '128K'로 확장되었다는 것을 보여줍니다. 이는 GPT 모델의 문맥 길이가 엄청나게 늘어났다는 의미입니다.
    * [01:08] 다음은 'Custom GPTs'입니다. 이것은 'Private or Public'으로 만들 수 있다고 되어 있네요. 개인적으로 사용하거나 공개적으로 공유할 수 있다는 뜻이겠죠.

    * [01:17] 'Multi Modal' 기능은 'Img, Video, Voice'를 지원한다고 명시되어 있습니다. 이미지, 비디오, 음성 등 다양한 형태의 데이터를 처리할 수 있다는 뜻입니다.
    * [01:25] 'JSON Mode'는 'Guaranteed'라고 되어 있습니다. 모델이 JSON 형식의 응답을 확실히 제공한다는 의미로 보입니다.
    * [01:32] 'Assistant API'는 'Developers' 즉 개발자들을 위한 새로운 API라고 합니다.
    * [01:37] 'Text 2 Speech' 기능은 현재 'Beta Release' 상태이며, 텍스트를 음성으로 변환해주는 기능입니다.

    * [01:45] 'Natural Voice Options'의 경우 '6 Voices'를 제공한다고 합니다. 여섯 가지의 자연스러운 음성 옵션을 선택할 수 있겠네요.
    * [01:52] 'GPT Store'는 'Revenue Shared'라고 되어 있어, 커스텀 GPT를 공유하고 수익을 나눌 수 있는 스토어 모델임을 알 수 있습니다.
    * [01:59] 'Conversation Threading'은 'Per Conversation'으로 관리된다고 합니다. 대화 단위로 스레드가 유지된다는 의미로 해석됩니다.

    * [02:07] 'File Uploading'은 'Multiple' 파일을 지원한다고 합니다. 여러 개의 파일을 한 번에 업로드할 수 있다는 뜻이죠.
    * [02:14] 가장 눈에 띄는 것 중 하나인 'API Price Reduction'은 무려 '2.5x - 3.5x'로 감소했다고 합니다. API 사용 비용이 크게 절감되었다는 것을 알 수 있습니다.
    * [02:24] 마지막으로 'Code Interpreter'와 'Function Calling'은 모두 'Built In'으로 표시되어 있습니다. 이 기능들이 이제 모델에 기본적으로 내장되어 제공된다는 의미입니다.

    * [02:35] 이미지의 맨 아래에는 'visit www.astratechz.com to build AI solutions'라는 문구가 있어, AI 솔루션 구축에 대한 아스트라테크즈의 서비스를 홍보하고 있습니다.

    * [02:45] 전체적으로 이 이미지는 2023년 11월 6일 OpenAI DevDay 이벤트의 핵심 발표 내용을 매우 깔끔하고 명확하게 요약하여 전달하고 있네요.

* --- 로컬 이미지 스트리밍 종료 ---


![사용 이미지](https://t3.ftcdn.net/jpg/03/77/33/96/360_F_377339633_Rtv9I77sSmSNcev8bEcnVxTHrXB4nRJ5.jpg) = 교재와 동일

* --- 온라인 이미지로 멀티모달 스트리밍 시작 ---
* [온라인 이미지 답변 (스트리밍)]:
* 이 온라인 이미지는 깨끗하고 현대적인 디자인의 데이터 테이블을 보여주고 있습니다. 전체적인 구성은 다음과 같습니다:

*   **배경 및 전체 레이아웃:**
    *   이미지의 주된 배경은 깨끗한 흰색입니다.
    *   테이블과 관련 텍스트는 중앙에 배치되어 있으며, 주변에 충분한 여백이 있어 시각적으로 답답하지 않습니다.

*   **테이블 제목:**
    *   테이블 상단에는 "TABLE 001: LOREM IPSUM DOLOR AMIS ENIMA ACCUMER TUNA"라는 제목이 있습니다.
    *   "TABLE 001:" 부분은 굵은 글씨로 강조되어 있으며, 나머지는 일반 텍스트입니다.
    *   전형적인 로렘 입숨(Lorem Ipsum) 스타일의 플레이스홀더 텍스트로 보입니다.

*   **테이블 구조 및 디자인:**
    *   **헤더 행 (상단):**
        *   첫 번째 헤더 셀은 비어 있는 밝은 하늘색의 직사각형 모양입니다.
        *   그 옆으로 "Loremiis", "Amis terim", "Gato lepis", "Tortores"라는 텍스트가 적힌 네 개의 밝은 하늘색 직사각형 셀이 이어집니다.
        *   모든 헤더 셀은 모서리가 둥글게 처리되어 있습니다. 텍스트는 흰색으로 표시되어 있습니다.
    *   **첫 번째 열 (왼쪽):**
        *   헤더 아래로 "Lorem dolor siamet", "Consecter odio", "Gatoque accums", "Sed hac enim rem", "Remps tortor just", "Klimas nsecter", "Babisak atoque accu", "Enim rem kos"와 같은 텍스트가 적힌 여덟 개의 밝은 하늘색 셀이 세로로 나열되어 있습니다.
        *   이 셀들 또한 모서리가 둥글게 처리되어 있으며, 텍스트는 흰색입니다. 이 열은 테이블의 각 행에 대한 레이블 역할을 합니다.
    *   **데이터 영역:**
        *   첫 번째 열 옆의 데이터 셀들은 흰색과 매우 밝은 회색 배경이 번갈아 가며 나타나는 스트라이프 패턴을 형성하고 있습니다. 이는 가독성을 높이는 데 도움이 됩니다.
        *   각 셀은 독립적인 직사각형 블록처럼 보이며, 셀 사이에는 얇은 간격이 있습니다.
        *   데이터는 주로 숫자로 구성되어 있으며, 각 열의 내용 유형은 다음과 같습니다:
            *   두 번째 열: 정수 (예: "8 288", "123")
            *   세 번째 열: 백분율 (예: "123 %", "87 %")
            *   네 번째 열: 상태 (예: "YES", "NO", "N/A")
            *   다섯 번째 열: 통화 금액 (예: "$89", "$129")
        *   모든 데이터 셀의 텍스트는 중앙에 정렬되어 있습니다.

*   **하단 설명 텍스트:**
    *   테이블 아래에는 작고 가는 글씨체로 된 "Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed non tellus natoque accumsan..."으로 시작하는 추가 텍스트 단락이 있습니다. 이 역시 로렘 입숨 플레이스홀더 텍스트이며, 테이블에 대한 추가 설명이나 정보 역할을 하는 것으로 보입니다.

*   **전반적인 스타일:**
    *   간결하고 깔끔한 미니멀리스트 디자인이 특징입니다.
    *   둥근 모서리와 부드러운 색상 팔레트(하늘색, 흰색, 연회색)가 사용자 친화적인 느낌을 줍니다.
    *   정보를 시각적으로 명확하고 이해하기 쉽게 제시하는 데 중점을 둔 UI/UX 디자인 요소로 보입니다.
  
* --- 온라인 이미지 스트리밍 종료 ---

* 모델 사용 완료

#### (5) System, User 프롬프트 수정

* 구글 공식 가이드 문서 [https://ai.google.dev/gemini-api/docs/migrate?hl=ko] 참고
* **GenAI SDK**로 이전 -> **업데이트된 클라이언트 아키텍쳐**로 작업

In [ ]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
from PIL import Image               # 이미지 처리를 위해 필요
import requests                     # URL에서 이미지를 가져오기 위해 필요
from io import BytesIO              # requests.get(stream=True)와 Image.open을 연결하기 위해 필요

# .env 파일 로드 (GEMINI_API_KEY 환경 변수 포함)
load_dotenv()

# 환경 변수에서 GEMINI_API_KEY 가져오기
# genai.Client()는 기본적으로 이 환경 변수를 찾음
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY 환경 변수가 설정되지 않았습니다.")

# --- 1. Gemini API 클라이언트 생성 ---
client = genai.Client(api_key=api_key)

# 사용할 모델
MODEL_NAME = 'gemini-1.5-flash' 

# --- 2. 이미지 로드 헬퍼 함수 정의 ---
def load_image_from_url(image_url):
    """온라인 URL에서 이미지를 다운로드하여 PIL.Image 객체로 로드합니다."""
    try:
        response = requests.get(image_url)
        response.raise_for_status() # HTTP 오류가 발생하면 예외 발생
        return Image.open(BytesIO(response.content)) # BytesIO를 사용하여 이미지 로드
    except Exception as e:
        raise ConnectionError(f"오류: 온라인 이미지 '{image_url}'을(를) 로드하는 데 실패했습니다: {e}")
    
# 사용자에게 보여줄 질문 (모델의 실제 입력 내용)
# user_prompt = """당신에게 주어진 표는 회사의 재무제표 입니다. 흥미로운 사실을 정리하여 답변하세요."""  -> 구체적이고 명확하게 질문하기
user_prompt =  """이 재무제표 이미지에서 2019년, 2018년, 2017년의 매출채권과 재고자산 값을 각각 알려주세요. 표 형식으로 정리해 주세요.""" 

# 분석할 재무제표 이미지 URL
ONLINE_IMAGE_URL = "https://storage.googleapis.com/static.fastcampus.co.kr/prod/uploads/202212/080345-661/kwon-01.png"

# --- 4. GenerateContentConfig 설정 ---
# 모델의 역할 (System Instruction)과 응답 생성 파라미터들을 config 객체 안에 정의합니다.
generation_config = types.GenerateContentConfig(
    system_instruction="""당신은 표(재무제표) 를 해석하는 금융 AI 어시스턴트 입니다. 
당신의 임무는 주어진 테이블 형식의 재무제표를 바탕으로 흥미로운 사실을 정리하여 친절하게 답변하는 것입니다.""",
    max_output_tokens=4096,     # 최대 출력 토큰 수 (재무제표 설명은 길 수 있으므로 충분히 확보) / 800 -> 2000 -> 4096 (응답 길이 확장)
    top_k=40,                   # 샘플링 시 상위 K개의 토큰만 고려 (다양성 제어) / 5 -> 40 (다양성 증가)
    top_p=0.9,                  # 누적 확률 P 이내의 토큰만 고려 (다양성 제어) / 0.8 -> 0.9 (다양성 증가)
    temperature=0.9,            # 응답의 창의성/무작위성 조절 (높을수록 창의적) / 0.8 -> 0.9 (창의성 증가)
    seed=42,                    # 재현 가능한 결과를 위한 시드값
    # response_mime_type='application/json', # JSON 응답이 필요하지 않으므로 주석 처리
    # stop_sequences=['\n'], # 스트리밍 시 답변이 중간에 끊길 수 있으므로 주석 처리
)

# --- 5. 모델 호출 (멀티모달 스트리밍) ---
print("\n--- 온라인 재무제표 이미지 분석 시작 ---")

try:
    # 재무제표 이미지 로드
    financial_statement_image = load_image_from_url(ONLINE_IMAGE_URL)

    # models.generate_content_stream 호출
    # contents: 이미지와 유저 프롬프트 (모델의 실제 입력)
    # config: 모델의 행동 방식과 생성 파라미터 (system_instruction 포함)
    response_chunks = client.models.generate_content_stream(
        model=MODEL_NAME,
        contents=[
            financial_statement_image,
            {'text': user_prompt} 
        ],
        config=generation_config,               # config 인자로 GenerateContentConfig 객체 전달
    )

    print("[재무제표 분석 답변 (스트리밍)]:")
    # --- 디버깅 코드 추가 ---
    all_response_text = ""
    for i, chunk in enumerate(response_chunks):
        print(f"\n--- 청크 {i+1} 시작 ---")
        print(f"청크 객체 타입: {type(chunk)}")
        print(f"청크 객체 내용: {chunk}") # 전체 청크 객체 출력

        if hasattr(chunk, 'candidates') and chunk.candidates:
            for cand_idx, candidate in enumerate(chunk.candidates):
                print(f"  후보 {cand_idx} - finish_reason: {candidate.finish_reason}")
                if hasattr(candidate, 'safety_ratings') and candidate.safety_ratings:
                    print(f"  후보 {cand_idx} - safety_ratings: {candidate.safety_ratings}")
                if hasattr(candidate, 'content') and candidate.content:
                    if hasattr(candidate.content, 'parts') and candidate.content.parts:
                        # content.parts에서 텍스트 부분만 추출
                        part_texts = [part.text for part in candidate.content.parts if hasattr(part, 'text')]
                        print(f"  후보 {cand_idx} - content.parts 텍스트: {''.join(part_texts)}")
        
        # 실제 출력될 텍스트 부분
        if chunk.text:
            print(f"청크 텍스트: {chunk.text}")
            all_response_text += chunk.text
        else:
            print("청크에 텍스트 없음 (chunk.text is None or empty)")
        print(f"--- 청크 {i+1} 종료 ---")
    # --- 디버깅 코드 끝 ---
    
    # 최종 응답이 None이 아닌지 확인
    if not all_response_text.strip(): # 공백만 있는 경우도 None으로 간주
        print("\n[경고: 모델이 텍스트를 생성하지 않았습니다. 위 디버깅 로그를 확인하세요.]")

    print("\n--- 재무제표 분석 종료 ---\n")

except ConnectionError as e:
    print(f"네트워크 오류: {e}")
except Exception as e:
    print(f"예상치 못한 오류 발생: {e}")

print("모델 사용 완료")


<small>

1. 사용 모델 
    - `gemini-2.5-flash` -> X (max-token 문제 발생)
    - `gemini-1.5-pro` -> X (호출 딜레이 문제 발생)
    - `gemini-1.5-flash` -> O 

---

![사용 이미지](https://storage.googleapis.com/static.fastcampus.co.kr/prod/uploads/202212/080345-661/kwon-01.png
) = 교재와 동일

---

2. 셀 출력

`(lc_env) ➜  01_2_GEMINI_TEST git:(main) ✗ python test_gemini_prompt.py`

* --- 온라인 재무제표 이미지 분석 시작 ---
* [재무제표 분석 답변 (스트리밍)]:

* --- 청크 1 시작 ---
* 청크 객체 타입: `<class 'google.genai.types.GenerateContentResponse'>`
* 청크 객체 내용: ```sdk_http_response=HttpResponse(
  headers=<dict len=11>
 ) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text='물'
      ),
    ],
    role='model'
  )
)] create_time=None response_id=None model_version='gemini-1.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=383,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=125
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=258
    ),
  ],
  total_token_count=383
) automatic_function_calling_history=None parsed=None```

  * 후보 0 - `finish_reason: None`
  * 후보 0 - `content.parts` 텍스트: 물
* 청크 텍스트: 물
* --- 청크 1 종료 ---

* --- 청크 2 시작 ---
* 청크 객체 타입: `<class 'google.genai.types.GenerateContentResponse'>`
* 청크 객체 내용: ```sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text='론입니다. 아래 표는 2017년, 2'
      ),
    ],
    role='model'
  )
)] create_time=None response_id=None model_version='gemini-1.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=383,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=125
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=258
    ),
  ],
  total_token_count=383
) automatic_function_calling_history=None parsed=None```
  * 후보 0 - `finish_reason: None`
  * 후보 0 - `content.parts` 텍스트: 론입니다. 아래 표는 2017년, 2
* 청크 텍스트: 론입니다. 아래 표는 2017년, 2
* --- 청크 2 종료 ---

* --- 청크 3 시작 ---
* 청크 객체 타입: `<class 'google.genai.types.GenerateContentResponse'>`
* 청크 객체 내용: ```sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text='018년, 2019년의 매출채권'
      ),
    ],
    role='model'
  )
)] create_time=None response_id=None model_version='gemini-1.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=383,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=125
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=258
    ),
  ],
  total_token_count=383
) automatic_function_calling_history=None parsed=None```
  * 후보 0 - `finish_reason: None`
  * 후보 0 - `content.parts` 텍스트: 018년, 2019년의 매출채권
* 청크 텍스트: 018년, 2019년의 매출채권
* --- 청크 3 종료 ---

* --- 청크 4 시작 ---
* 청크 객체 타입: `<class 'google.genai.types.GenerateContentResponse'>`
* 청크 객체 내용: ```sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text=""" 및 재고자산을 보여줍니다.

| 연도 | 매출채권 (백만원) | 재고자산 (백"""
      ),
    ],
    role='model'
  )
)] create_time=None response_id=None model_version='gemini-1.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=383,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=125
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=258
    ),
  ],
  total_token_count=383
) automatic_function_calling_history=None parsed=None```
  * 후보 0 - `finish_reason: None`
  * 후보 0 - `content.parts` 텍스트:  및 재고자산을 보여줍니다.

* | 연도 | 매출채권 (백만원) | 재고자산 (백
* 청크 텍스트:  및 재고자산을 보여줍니다.

* | 연도 | 매출채권 (백만원) | 재고자산 (백
* --- 청크 4 종료 ---

* --- 청크 5 시작 ---
* 청크 객체 타입: `<class 'google.genai.types.GenerateContentResponse'>`
* 청크 객체 내용: ```sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""만원) |
|---|---|---|
| 2017 | 3,781,000 | 2,07"""
      ),
    ],
    role='model'
  )
)] create_time=None response_id=None model_version='gemini-1.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=383,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=125
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=258
    ),
  ],
  total_token_count=383
) automatic_function_calling_history=None parsed=None```
  * 후보 0 - `finish_reason: None`
  * 후보 0 - `content.parts` 텍스트: 만원) |
* |---|---|---|
* | 2017 | 3,781,000 | 2,07
* 청크 텍스트: 만원) |
* |---|---|---|
* | 2017 | 3,781,000 | 2,07
* --- 청크 5 종료 ---

* --- 청크 6 시작 ---
* 청크 객체 타입: `<class 'google.genai.types.GenerateContentResponse'>`
* 청크 객체 내용: ```sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""4,555 |
| 2018 | 4,004,920 | 2,426,364 |
| 2019 | 3,9"""
      ),
    ],
    role='model'
  )
)] create_time=None response_id=None model_version='gemini-1.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=383,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=125
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=258
    ),
  ],
  total_token_count=383
) automatic_function_calling_history=None parsed=None```
  * 후보 0 - `finish_reason: None`
  * 후보 0 - `content.parts` 텍스트: 4,555 |
* | 2018 | 4,004,920 | 2,426,364 |
* | 2019 | 3,9
* 청크 텍스트: 4,555 |
* | 2018 | 4,004,920 | 2,426,364 |
* | 2019 | 3,9
* --- 청크 6 종료 ---

* --- 청크 7 시작 ---
* 청크 객체 타입: `<class 'google.genai.types.GenerateContentResponse'>`
* 청크 객체 내용: ```sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""81,935 | 2,670,294 |


**추가 정보:**

* **매출채권:** 2018년에 최고치를 기록했지만,"""
      ),
    ],
    role='model'
  )
)] create_time=None response_id=None model_version='gemini-1.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=383,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=125
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=258
    ),
  ],
  total_token_count=383
) automatic_function_calling_history=None parsed=None```
  * 후보 0 - `finish_reason: None`
  * 후보 0 - `content.parts` 텍스트: 81,935 | 2,670,294 |


* **추가 정보:**

  * **매출채권:** 2018년에 최고치를 기록했지만,
  * 청크 텍스트: 81,935 | 2,670,294 |


* **추가 정보:**

  * **매출채권:** 2018년에 최고치를 기록했지만,
* --- 청크 7 종료 ---

* --- 청크 8 시작 ---
* 청크 객체 타입: `<class 'google.genai.types.GenerateContentResponse'>`
* 청크 객체 내용: ```sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text=""" 2019년에는 약간 감소했습니다. 이는 회사의 판매 및 회수 프로세스에 대한 추가 분석이 필요할 수 있음을 시사합니다.
* **재고자산:** 2017년부터 2019년까지 """
      ),
    ],
    role='model'
  )
)] create_time=None response_id=None model_version='gemini-1.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=383,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=125
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=258
    ),
  ],
  total_token_count=383
) automatic_function_calling_history=None parsed=None```
  * 후보 0 - `finish_reason: None`
  * 후보 0 - `content.parts` 텍스트:  2019년에는 약간 감소했습니다. 이는 회사의 판매 및 회수 프로세스에 대한 추가 분석이 필요할 수 있음을 시사합니다.
* **재고자산:** 2017년부터 2019년까지 
* 청크 텍스트:  2019년에는 약간 감소했습니다. 이는 회사의 판매 및 회수 프로세스에 대한 추가 분석이 필요할 수 있음을 시사합니다.
* **재고자산:** 2017년부터 2019년까지 
* --- 청크 8 종료 ---

* --- 청크 9 시작 ---
* 청크 객체 타입: `<class 'google.genai.types.GenerateContentResponse'>`
* 청크 객체 내용: ```sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""꾸준히 증가하고 있습니다. 이는 회사의 생산량 증가 또는 재고 관리 전략 변경을 나타낼 수 있습니다.
이러한 경향에 대한 더 나은 이해를 위해서는 추가적인 재무 정보와 비즈니스 컨텍스트가 필요합니다."""
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>
)] create_time=None response_id=None model_version='gemini-1.5-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=320,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=320
    ),
  ],
  prompt_token_count=381,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=123
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=258
    ),
  ],
  total_token_count=701
) automatic_function_calling_history=None parsed=None```
  * 후보 0 - `finish_reason: FinishReason.STOP`
  * 후보 0 - `content.parts` 텍스트: 꾸준히 증가하고 있습니다. 이는 회사의 생산량 증가 또는 재고 관리 전략 변경을 나타낼 수 있습니다.

* 이러한 경향에 대한 더 나은 이해를 위해서는 추가적인 재무 정보와 비즈니스 컨텍스트가 필요합니다.
* 청크 텍스트: 꾸준히 증가하고 있습니다. 이는 회사의 생산량 증가 또는 재고 관리 전략 변경을 나타낼 수 있습니다.

* 이러한 경향에 대한 더 나은 이해를 위해서는 추가적인 재무 정보와 비즈니스 컨텍스트가 필요합니다.
* --- 청크 9 종료 ---

* --- 재무제표 분석 종료 ---

* 모델 사용 완료

---

3. 실제 출력값

- 물론입니다. 아래 표는 2017년, 2018년, 2019년의 매출채권 및 재고자산을 보여줍니다.

  - | 연도 | 매출채권 (백만원) | 재고자산 (백만원) |
  - |------|--------------|---------------|
  - | 2017 |   3,781,000  |   2,074,555   |
  - | 2018 |   4,004,920  |   2,426,364   |
  - | 2019 |   3,981,935  |   2,670,294   |

- **추가 정보:**

  - **매출채권:** 2018년에 최고치를 기록했지만, 2019년에는 약간 감소했습니다. 이는 회사의 판매 및 회수 프로세스에 대한 추가 분석이 필요할 수 있음을 시사합니다.
  - **재고자산:** 2017년부터 2019년까지 꾸준히 증가하고 있습니다. 이는 회사의 생산량 증가 또는 재고 관리 전략 변경을 나타낼 수 있습니다.

- 이러한 경향에 대한 더 나은 이해를 위해서는 추가적인 재무 정보와 비즈니스 컨텍스트가 필요합니다.
